In [1]:
using JuMP
using GLPK
using ECOS
using MAT
using SparseArrays
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using DelimitedFiles
using COSMO
using DataFrames
using CSV
using IJulia
IJulia.installkernel("Julia nodeps", "--depwarn=no")
using Suppressor

┌ Info: Installing Julia nodeps kernelspec in /home/ehsan/.local/share/jupyter/kernels/julia-nodeps-1.6
└ @ IJulia /home/ehsan/.julia/packages/IJulia/e8kqU/deps/kspec.jl:94


In [126]:
X = Array(DataFrame(CSV.File("output.txt", delim=",",header=false)));
hit = vec(DataFrame(CSV.File("hit.csv", delim=",",header=false))[:,1]);
miss = vec(DataFrame(CSV.File("miss.csv", delim=",",header=false))[:,1]);


In [127]:
pref = zeros(size(X)[1])
for i in 1:size(X)[1]
    pref[i] = sum(X[i,:] .> 2e-5) + sum(X[i,:] .< -2e-5)
end
minimum(pref)

0.0

In [232]:
vars = matread("R5T3.mat")
S = vars["S"]
U = vars["U"]
L = vars["L"]
K = vars["K"]
n,d = size(S)
print(size(S))
print(size(L))

n,d = size(S) # n=10 d=100
d,k = size(L) # k = 5

print("done")

target = size(L)[2] - K ;

(15638, 28301)(28301, 200)done

In [130]:
X = (x->round.(x,digits=8)).(X);

In [226]:
function f_zero_rows(X)
    zero_rows = 0
    for i in 1:size(X)[1]
        zero_rows += Int16( (sum( X[i,:] .>= 2e-5 ) + sum( X[i,:] .<= -2e-5 )) == 0)
    end
    return zero_rows
end;

In [227]:
function f_zero_cols(Z)
    zero_cols = 0
    for i in 1:size(Z)[2]
        zero_cols += Int16( (sum( Z[:,i] .> 2e-5 ) + sum( Z[:,i] .< -2e-5 )) == 0)
    end
    return zero_cols
end;

In [1]:
tresh = 1
while true
    
    function myCondition(y)
        return y == tresh
    end
    pot_ind = findall(myCondition,pref);
    
    
    X_p = copy(X)
    for i in pot_ind
        X[i,:] .= 0
        Z = S*X;
        bad = sum(vec(X) .> vec(U)+ones(size(vec(U))[1])*2e-5) + sum(vec(X) .< vec(L)-ones(size(vec(U))[1])*2e-5)
        if( ((f_zero_cols(Z))<target) | bad >0 )
            X = copy(X_p)
        else
            X_p = copy(X)
        end
    end
    Z = S*X;
    z_rows = f_zero_rows(X)
    println("zero rows: ", z_rows)
    writedlm("output_$z_rows.txt", Matrix(reshape(value.(X),(d,k))), ',')
    tresh += 1
end